# 2 Flood CNN's for single region Årdal and Multi region

In [1]:
import numpy as np
import rasterio
from rasterio.windows import Window
import geopandas as gpd
from pathlib import Path
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
from tqdm import tqdm
from pathlib import Path

# Årdal

In [ ]:
from pathlib import Path
from shapely.geometry import Point
import numpy as np
import pandas as pd
import rasterio
from rasterio.windows import Window
from tqdm import tqdm
import geopandas as gpd
LABELS_PATH = Path("data/processed/features/houses_ardal_flom_labels.parquet")
RASTER_DIR  = Path("data/derived/rasters")
OUT_DIR     = Path("data/derived/dtm_features_ardal10m")
OUT_DIR.mkdir(parents=True, exist_ok=True)

DTM_PATH      = RASTER_DIR / "ardal_dtm10_25833.tif"
SLOPE_PATH    = RASTER_DIR / "ardal_slope_deg_25833.tif"
ASPECT_PATH   = RASTER_DIR / "ardal_aspect_deg_25833.tif"
CURV_PATH     = RASTER_DIR / "ardal_curvature_25833.tif"
FLOWACC_PATH  = RASTER_DIR / "ardal_flowacc_25833.tif"
TWI_PATH      = RASTER_DIR / "ardal_twi.tif"
HOUSES_GPKG = Path("../data/processed/vector/exposure/houses_ardal_flom.gpkg")
HOUSES_LAYER = "houses_flom"
PATCH_SIZE_M = 200.0  # 200x200 m around each house

def build_house_stacks():
    df = gpd.read_file(HOUSES_GPKG, layer=HOUSES_LAYER).to_crs(25833)
    df.set_geometry("geometry")
    df = df[df["hazard_class_flom"].notna()].copy()
    df["bygningsnummer"] = df["bygningsnummer"].astype(int)

    with rasterio.open(DTM_PATH) as dem_src, \
         rasterio.open(SLOPE_PATH) as slope_src, \
         rasterio.open(ASPECT_PATH) as aspect_src, \
         rasterio.open(CURV_PATH) as curv_src, \
         rasterio.open(FLOWACC_PATH) as fa_src, \
         rasterio.open(TWI_PATH) as twi_src:

        res_x, res_y = dem_src.res  # should be ~10,10 for DTM10
        assert abs(res_x - res_y) < 1e-6
        res = res_x

        half_pixels = int((PATCH_SIZE_M / 2) / res)  
        patch_pixels = half_pixels * 2               

        for _, row in tqdm(df.iterrows(), total=len(df), desc="Extracting house stacks"):
            bid = int(row["bygningsnummer"])
            out_path = OUT_DIR / f"house_{bid}.npy"
            if out_path.exists():
                continue

            geom = row.geometry

            # Convert coord to raster indices
            row_idx, col_idx = dem_src.index(geom.x, geom.y)

            window = Window(
                col_idx - half_pixels,
                row_idx - half_pixels,
                patch_pixels,
                patch_pixels,
            )

            dem_patch    = dem_src.read(1, window=window)
            slope_patch  = slope_src.read(1, window=window)
            aspect_patch = aspect_src.read(1, window=window)
            curv_patch   = curv_src.read(1, window=window)
            fa_patch     = fa_src.read(1, window=window)
            twi_patch    = twi_src.read(1, window=window)

            if dem_patch.shape != (patch_pixels, patch_pixels):
                # skip houses near the raster edge
                continue

            stack = np.stack(
                [dem_patch, slope_patch, aspect_patch, curv_patch, fa_patch, twi_patch],
                axis=0,
            ).astype("float32")

            # basic NaN handling
            stack = np.where(np.isfinite(stack), stack, np.nan)
            for c in range(stack.shape[0]):
                chan = stack[c]
                med = np.nanmedian(chan)
                stack[c] = np.where(np.isnan(chan), med, chan)

            stack


In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

DATA_ROOT = Path("../data")
PATCH_DIR = DATA_ROOT / "derived" / "dtm_patches"
FEATURE_DIR = Path("data/derived/dtm_features_ardal10m")
LABELS_PATH = DATA_ROOT / "processed" / "features" / "houses_ardal_flom_labels.parquet"
STATS_PATH  = FEATURE_DIR / "channel_stats.npz"
class FloodPatchDataset(Dataset):
    def __init__(self, split: str = "train", normalize: bool = True):
        df = pd.read_parquet(LABELS_PATH)
        df = df[df["hazard_class_flom"].notna()].copy()
        df["hazard_class_flom"] = df["hazard_class_flom"].astype(int)

        if "split" in df.columns:
            df = df[df["split"] == split]

        df["feature_path"] = df["bygningsnummer"].astype(int).apply(
            lambda bid: FEATURE_DIR / f"house_{bid}.npy"
        )

        mask = df["feature_path"].apply(lambda p: p.exists())
        missing = (~mask).sum()
        if missing > 0:
            print(f"[WARN] {missing} feature stacks missing, dropping those rows")
        df = df[mask].reset_index(drop=True)

        self.df = df
        self.normalize = normalize

        stats = np.load(STATS_PATH)
        self.channel_mean = stats["mean"]  # (C,)
        self.channel_std  = stats["std"]   # (C,)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        arr = np.load(row["feature_path"])  # (C,H,W)
        if self.normalize:
            mean = self.channel_mean[:, None, None]
            std = self.channel_std[:, None, None]
            arr = (arr - mean) / (std + 1e-6)
        x = torch.from_numpy(arr).float()
        y = torch.tensor(row["hazard_class_flom"], dtype=torch.long)
        return x, y


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SmallFloodCNN(nn.Module):
    def __init__(self, in_channels: int = 6, num_classes: int = 3):
        super().__init__()

        self.features = nn.Sequential(
            # Block 1
            nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  

            # Block 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )

        
        self.gap = nn.AdaptiveAvgPool2d(1)

        self.classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.gap(x)
        x = torch.flatten(x, 1)  # (N,256)
        logits = self.classifier(x)
        return logits


In [ ]:
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_ds = FloodPatchDataset(split="train", normalize=True)
val_ds   = FloodPatchDataset(split="val", normalize=True)

print("Train size:", len(train_ds), "Val size:", len(val_ds))

train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds, batch_size=8, shuffle=False, num_workers=0)

# --- model / loss ---
model = SmallFloodCNN(in_channels=6, num_classes=3).to(device)

class_counts = train_ds.df["hazard_class_flom"].value_counts()
weights = torch.ones(3)
if 0 in class_counts.index and 2 in class_counts.index:
    weights[2] = class_counts[0] / class_counts[2]  # upweight rare high class
criterion = nn.CrossEntropyLoss(weight=weights.to(device))

optimizer = Adam(model.parameters(), lr=1e-3)

def train_one_epoch(epoch):
    model.train()
    running_loss = 0.0
    for i, (x, y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch}: train_loss={running_loss/len(train_loader):.4f}")

def evaluate():
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            running_loss += loss.item()

            preds = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    acc = correct / total if total > 0 else 0.0
    print(f"  val_loss={running_loss/len(val_loader):.4f}, val_acc={acc:.3f}")
    return acc

for epoch in range(1, 21):
    train_one_epoch(epoch)
    evaluate()


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

def evaluate_and_report(loader, model, device):
    model.eval()
    all_y = []
    all_pred = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            preds = logits.argmax(dim=1)
            all_y.append(y.cpu().numpy())
            all_pred.append(preds.cpu().numpy())

    y_true = np.concatenate(all_y)
    y_pred = np.concatenate(all_pred)

    print("Confusion matrix (rows=true, cols=pred, order [0,1,2]):")
    print(confusion_matrix(y_true, y_pred, labels=[0,1,2]))
    print()
    print(classification_report(y_true, y_pred, labels=[0,1,2]))

evaluate_and_report(val_loader, model, device)


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import geopandas as gpd
import torch

DATA_ROOT = Path("../data")
FEATURE_DIR = Path("data/derived/dtm_features_ardal10m")
LABELS_PATH = DATA_ROOT / "processed" / "features" / "houses_ardal_flom_labels.parquet"
STATS_PATH  = FEATURE_DIR / "channel_stats.npz"

HOUSES_GPKG = Path("../data/processed/vector/exposure/houses_ardal_flom.gpkg")
HOUSES_LAYER = "houses_flom"

OUT_GPKG = Path("data/processed/vector/exposure/houses_ardal_flom_preds.gpkg")
OUT_LAYER = "houses_flom_preds_clean"


def build_preds_gpkg(model, device):
    df = pd.read_parquet(LABELS_PATH)
    df = df[df["hazard_class_flom"].notna()].copy()
    df["hazard_class_flom"] = df["hazard_class_flom"].astype(int)
    df["feature_path"] = df["bygningsnummer"].astype(int).apply(
        lambda bid: FEATURE_DIR / f"house_{bid}.npy"
    )
    df = df[df["feature_path"].apply(lambda p: p.exists())].copy()

    stats = np.load(STATS_PATH)
    mean = stats["mean"]  # (C,)
    std  = stats["std"]   # (C,)
    mean_b = mean[:, None, None]
    std_b  = std[:, None, None]

    all_pred_class = []
    all_p_high = []

    model.eval()
    with torch.no_grad():
        for _, row in df.iterrows():
            arr = np.load(row["feature_path"])  # raw (C,H,W)
            # same normalization as in FloodPatchDataset
            arr = (arr - mean_b) / (std_b + 1e-6)

            x = torch.from_numpy(arr).unsqueeze(0).float().to(device)
            logits = model(x)
            probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

            pred_cls = int(np.argmax(probs))
            p_high = float(probs[2])  # prob of class 2

            all_pred_class.append(pred_cls)
            all_p_high.append(p_high)

    df["pred_class_flom"] = all_pred_class
    df["pred_p_high"] = all_p_high

    houses = gpd.read_file(HOUSES_GPKG, layer=HOUSES_LAYER).to_crs(25833)
    houses = houses[houses["hazard_class_flom"].notna()].copy()
    houses["hazard_class_flom"] = houses["hazard_class_flom"].astype(int)
    merged = houses.merge(
        df[["bygningsnummer", "pred_class_flom", "pred_p_high"]],
        on="bygningsnummer",
        how="left",
    )

    OUT_GPKG.parent.mkdir(parents=True, exist_ok=True)
    merged.to_file(OUT_GPKG, layer=OUT_LAYER, driver="GPKG")
    print("Wrote predictions to", OUT_GPKG, "layer", OUT_LAYER)

    mask = merged["pred_class_flom"].notna()
    acc = (merged.loc[mask, "pred_class_flom"] == merged.loc[mask, "hazard_class_flom"]).mean()
    print(f"Overall accuracy on all labelled houses: {acc:.3f} (should be close-ish to your val_acc)")


build_preds_gpkg(model, device)


# Multi Regional

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Callable, Any

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.windows import Window
import torch
from torch.utils.data import Dataset


def subset_cols(df: gpd.GeoDataFrame, keep_cols: List[str]):
    cols = [c for c in keep_cols if c in df.columns]
    if "geometry" in df.columns and "geometry" not in cols:
        cols.append("geometry")
    return df[cols]


def ensure_crs(gdf: gpd.GeoDataFrame, target_crs: str):
    if gdf.crs is None or gdf.crs.to_string() != target_crs:
        return gdf.to_crs(target_crs)
    return gdf


def simple_nan_fill_patch(patch: torch.Tensor) -> torch.Tensor:
    """
    Patch: (C, H, W). Fill NaNs per-channel with that channel's median.
    Very close to what you did earlier, but fast + reusable.
    """
    x = patch.clone()
    c = x.shape[0]
    for i in range(c):
        chan = x[i]
        mask = torch.isfinite(chan)
        if mask.any():
            med = chan[mask].median()
            chan = torch.where(mask, chan, med)
        else:
            chan = torch.zeros_like(chan)
        x[i] = chan
    return x


class HouseRasterDataset(Dataset):
    """
    On-the-fly multi-channel raster patches + tabular + label.

    houses_gdf must already include geometry + label + tabular columns.
    """

    def __init__(
        self,
        houses_gdf: gpd.GeoDataFrame,
        raster_paths: Dict[str, Path],
        patch_size_pixels: int,
        tabular_cols: List[str],
        label_col: str,
        house_id_col: str = "bygningsnummer",
        target_crs: str = "EPSG:25833",
        transform_fn: Optional[Callable[[torch.Tensor], torch.Tensor]] = None,
        fill_edges_with_nan: bool = True,
    ):
        houses_gdf = ensure_crs(houses_gdf, target_crs)

        self.houses = houses_gdf.reset_index(drop=True)
        self.raster_keys = list(raster_paths.keys())
        self.rasters = {k: rasterio.open(str(p)) for k, p in raster_paths.items()}

        self.patch_size = int(patch_size_pixels)
        self.half = self.patch_size // 2

        self.tabular_cols = tabular_cols
        self.label_col = label_col
        self.house_id_col = house_id_col
        self.transform_fn = transform_fn
        self.fill_edges_with_nan = fill_edges_with_nan

        # sanity: check all rasters share CRS/resolution-ish
        ref = next(iter(self.rasters.values()))
        self.ref_crs = ref.crs
        self.ref_res = ref.res

    def __len__(self):
        return len(self.houses)

    def close(self):
        for src in self.rasters.values():
            try:
                src.close()
            except Exception:
                pass

    def _window_for_point(self, src: rasterio.io.DatasetReader, x: float, y: float):
        row_idx, col_idx = src.index(x, y)
        return Window(
            col_off=col_idx - self.half,
            row_off=row_idx - self.half,
            width=self.patch_size,
            height=self.patch_size,
        )

    def _read_multichannel_patch(self, geom) -> torch.Tensor:
        """
        Returns (C,H,W) float32 tensor.
        Uses boundless read to avoid edge crashes.
        """
        patches = []
        for k in self.raster_keys:
            src = self.rasters[k]
            win = self._window_for_point(src, geom.x, geom.y)

            arr = src.read(
                1,
                window=win,
                boundless=True,
                fill_value=np.nan if self.fill_edges_with_nan else 0,
            ).astype("float32")

            patches.append(arr)

        stack = np.stack(patches, axis=0).astype("float32")
        return torch.from_numpy(stack)

    def __getitem__(self, idx):
        row = self.houses.iloc[idx]
        geom = row.geometry

        patch = self._read_multichannel_patch(geom)

        # optional quick NaN handling
        patch = simple_nan_fill_patch(patch)

        if self.transform_fn:
            patch = self.transform_fn(patch)

        tab_vals = row[self.tabular_cols].values.astype(np.float32)
        tab_tensor = torch.from_numpy(tab_vals)

        label = int(row[self.label_col])
        label_tensor = torch.tensor(label, dtype=torch.long)

        house_id = row.get(self.house_id_col, idx)

        return {
            "patch": patch,
            "tabular": tab_tensor,
            "label": label_tensor,
            "house_id": house_id,
        }

def compute_channel_stats(
    houses_gdf: gpd.GeoDataFrame,
    raster_paths: Dict[str, Path],
    patch_size_pixels: int,
    target_crs: str = "EPSG:25833",
    sample_n: int = 2000,
    seed: int = 42,
    out_path: Optional[str | Path] = None,
):
    houses_gdf = ensure_crs(houses_gdf, target_crs)

    if len(houses_gdf) == 0:
        raise ValueError("houses_gdf is empty")

    rng = np.random.default_rng(seed)
    n = min(sample_n, len(houses_gdf))
    sample_idx = rng.choice(len(houses_gdf), size=n, replace=False)

    raster_keys = list(raster_paths.keys())
    rasters = {k: rasterio.open(str(p)) for k, p in raster_paths.items()}

    half = patch_size_pixels // 2
    c = len(raster_keys)

    # Welford stats per channel
    count = np.zeros(c, dtype=np.int64)
    mean = np.zeros(c, dtype=np.float64)
    m2 = np.zeros(c, dtype=np.float64)

    def window_for_point(src, x, y):
        row_idx, col_idx = src.index(x, y)
        return Window(col_idx - half, row_idx - half, patch_size_pixels, patch_size_pixels)

    try:
        for i in sample_idx:
            geom = houses_gdf.iloc[i].geometry

            for ci, k in enumerate(raster_keys):
                src = rasters[k]
                win = window_for_point(src, geom.x, geom.y)

                arr = src.read(
                    1,
                    window=win,
                    boundless=True,
                    fill_value=np.nan,
                ).astype("float32")

                vals = arr[np.isfinite(arr)]
                if vals.size == 0:
                    continue

                # update channel stats with batch of values
                for v in vals:
                    count[ci] += 1
                    delta = v - mean[ci]
                    mean[ci] += delta / count[ci]
                    delta2 = v - mean[ci]
                    m2[ci] += delta * delta2

    finally:
        for src in rasters.values():
            try:
                src.close()
            except Exception:
                pass

    # avoid divide-by-zero
    var = np.where(count > 1, m2 / (count - 1), 0.0)
    std = np.sqrt(var)

    stats = {
        "keys": np.array(raster_keys),
        "mean": mean.astype("float32"),
        "std": std.astype("float32"),
        "count": count,
        "patch_size_pixels": np.array([patch_size_pixels]),
        "sample_n": np.array([n]),
    }

    if out_path is not None:
        out_path = Path(out_path)
        out_path.parent.mkdir(parents=True, exist_ok=True)
        np.savez(out_path, **stats)
        print("Wrote channel stats:", out_path)
        print("keys:", raster_keys)
        print("mean:", stats["mean"])
        print("std:", stats["std"])

    return stats

class NormalizeChannels:
    def __init__(self, stats_path: str | Path):
        d = np.load(stats_path, allow_pickle=True)
        self.keys = list(d["keys"])
        self.mean = torch.tensor(d["mean"], dtype=torch.float32).view(-1, 1, 1)
        self.std = torch.tensor(d["std"], dtype=torch.float32).view(-1, 1, 1)
        self.std = torch.where(self.std == 0, torch.ones_like(self.std), self.std)

    def __call__(self, patch: torch.Tensor):
        # patch (C,H,W)
        return (patch - self.mean) / self.std


In [ ]:
from sklearn.model_selection import train_test_split


regions = ["sogn", "lillehammer", "oslo"]

tab_cols = [
    "dist_to_river",
    "dist_to_lake",
    "dist_to_hyd",
    "QNormalOppstrm_Mm3Aar",
    "QNormal_lskm2",
    "QNormal_Mm3Aar",
    "dist_to_trigger_point",
    "dist_to_runout_area",
    "dist_to_runout_point",
    "dist_to_landslide_event",
    "dist_to_source_area"
    # add your other numeric cols here
]
def default_raster_paths_for_region(
    region: str,
    processed_raster_dir: str | Path,
) -> Dict[str, Path]:
    """
    Simple convention-based builder.
    Adjust names if your files differ.
    """
    processed_raster_dir = Path(processed_raster_dir)

    return {
        "dtm": processed_raster_dir / f"{region}_dtm10_filled.tif",
        "slope": processed_raster_dir / f"{region}_slope_deg.tif",
        "aspect": processed_raster_dir / f"{region}_aspect_deg.tif",
        "curv": processed_raster_dir / f"{region}_curvature.tif",
        "flowacc": processed_raster_dir / f"{region}_flowacc_d8.tif",
        "twi": processed_raster_dir / f"{region}_twi.tif",
    }
def split_houses_train_val(
    houses: gpd.GeoDataFrame,
    label_col: str,
    val_size: float = 0.2,
    seed: int = 42,
    stratify: bool = True,
):
    if len(houses) == 0:
        raise ValueError("No houses to split.")

    if label_col not in houses.columns:
        raise ValueError(f"Label col '{label_col}' not found in houses.")

    if train_test_split is None:
        # simple fallback without sklearn
        houses = houses.sample(frac=1.0, random_state=seed).copy()
        n_val = int(len(houses) * val_size)
        val = houses.iloc[:n_val].copy()
        train = houses.iloc[n_val:].copy()
        return train, val

    y = houses[label_col].astype("int64", errors="ignore")

    # If stratify requested but classes too tiny, fallback gracefully.
    strat = y if stratify else None
    try:
        train_idx, val_idx = train_test_split(
            houses.index.to_numpy(),
            test_size=val_size,
            random_state=seed,
            stratify=strat,
        )
    except Exception:
        train_idx, val_idx = train_test_split(
            houses.index.to_numpy(),
            test_size=val_size,
            random_state=seed,
            stratify=None,
        )

    train = houses.loc[train_idx].copy()
    val = houses.loc[val_idx].copy()
    return train, val

def make_region_dataset(
    region: str,
    houses_gpkg: str | Path,
    houses_layer: str,
    tabular_cols: List[str],
    label_col="hazard_class_flom",
    patch_size_pixels = 20,
    house_id_col: str = "bygningsnummer",
    target_crs: str = "EPSG:25833",
    raster_paths: Optional[Dict[str, Path]] = None,
    processed_raster_dir: str | Path = "../processed/rasters",
    compute_stats: bool = True,
    stats_out_dir: str | Path = "../processed/channel_stats",
    stats_sample_n: int = 1500,
    normalize: bool = True,
    val_size: float = 0.2,
    stratify: bool = True,
    seed: int = 42,
):
    """
    Returns:
        dataset, stats_path (or None), houses_gdf
    """
    # ------------------
    # 1) Load houses
    # ------------------
    houses = gpd.read_file(houses_gpkg, layer=houses_layer)
    houses = ensure_crs(houses, target_crs)
    f_links = pd.read_parquet(f"../processed/links/flood_links_{region}.parquet")
    landslide_links = pd.read_parquet(f"../processed/links/landslide_links_closest_{region}.parquet")
    hydro_links = pd.read_parquet(f"../processed/links/hydro_links_{region}.parquet")
    
    # add empty tabular columns to houses
    houses = subset_cols(houses, [house_id_col, label_col] + tabular_cols)
    
    f_links = f_links[[c for c in f_links.columns if c in (tabular_cols + [house_id_col, label_col])]].copy()
    landslide_links = landslide_links[[c for c in landslide_links.columns if c in (tabular_cols + [house_id_col])]].copy()
    hydro_links = hydro_links[[c for c in hydro_links.columns if c in (tabular_cols + [house_id_col])]].copy()
    
    houses = houses.merge(f_links, on=house_id_col, how="left", suffixes=("", "_flood"))
    houses = houses.merge(landslide_links, on=house_id_col, how="left", suffixes=("", "_landslide"))
    houses = houses.merge(hydro_links, on=house_id_col, how="left", suffixes=("", "_hydro"))
    
    if label_col not in houses.columns:
        raise ValueError(f"label_col='{label_col}' not found in houses layer")

    houses = houses[houses[label_col].notna()].copy()
    if len(houses) == 0:
        raise ValueError(f"No houses with non-null {label_col} for region={region}")

    # ------------------
    # 2) Resolve raster paths
    # ------------------
    if raster_paths is None:
        raster_paths = default_raster_paths_for_region(
            region=region,
            processed_raster_dir=processed_raster_dir,
        )

    # quick existence check
    missing = [k for k, p in raster_paths.items() if not Path(p).exists()]
    if missing:
        raise FileNotFoundError(
            f"Missing raster files for region={region}: {missing}\n"
            f"Paths: { {k: str(v) for k, v in raster_paths.items()} }"
        )

    houses_train, houses_val = split_houses_train_val(
        houses=houses,
        label_col=label_col,
        val_size=val_size,
        seed=seed,
        stratify=stratify,
    )
    # ------------------
    # 3) Compute / locate stats
    # ------------------
    stats_out_dir = Path(stats_out_dir)
    stats_out_dir.mkdir(parents=True, exist_ok=True)
    stats_path = stats_out_dir / f"{region}_channel_stats.npz"

    if not stats_path.exists():
        compute_channel_stats(
            houses_gdf=houses_train,
            raster_paths=raster_paths,
            patch_size_pixels=patch_size_pixels,
            target_crs=target_crs,
            sample_n=stats_sample_n,
            seed=seed,
            out_path=stats_path,
        )

    # ------------------
    # 4) Optional normalization transform
    # ------------------
    transform_fn = None
    if normalize:
        if stats_path is None:
            raise ValueError("normalize=True but compute_stats=False and no stats_path available")
        transform_fn = NormalizeChannels(stats_path)

    # ------------------
    # 5) Build dataset
    # ------------------
    train_ds = HouseRasterDataset(
        houses_gdf=houses_train,
        raster_paths=raster_paths,
        patch_size_pixels=patch_size_pixels,
        tabular_cols=tabular_cols,
        label_col=label_col,
        house_id_col=house_id_col,
        target_crs=target_crs,
        transform_fn=transform_fn,
    )

    val_ds = HouseRasterDataset(
        houses_gdf=houses_val,
        raster_paths=raster_paths,
        patch_size_pixels=patch_size_pixels,
        tabular_cols=tabular_cols,
        label_col=label_col,
        house_id_col=house_id_col,
        target_crs=target_crs,
        transform_fn=transform_fn,
    )

    return train_ds, val_ds, stats_path, houses_train, houses_val


In [ ]:
regions = ["sogn", "lillehammer", "oslo"]

tab_cols = [
    "dist_to_river",
    "dist_to_lake",
    "dist_to_hyd",
]

def build_all_regions_train(
    houses_gpkg_tpl: str | Path,
    houses_layer_tpl: str,
    batch_size: int = 8,
    num_workers: int = 0,
    pin_memory: bool = False,
    shuffle_train: bool = True,
    raster_paths_by_region: Optional[Dict[str, Dict[str, Path]]] = None,
    ) -> Dict[str, Dict[str, Any]]:
    results: Dict[str, Dict[str, Any]] = {}
    regions = ["sogn", "lillehammer", "oslo"]

    tab_cols = [
        "dist_to_river",
        "dist_to_lake",
        "dist_to_hyd",
        "QNormalOppstrm_Mm3Aar",
        "QNormal_lskm2",
        "QNormal_Mm3Aar",
        "dist_to_trigger_point",
        "dist_to_runout_area",
        "dist_to_runout_point",
        "dist_to_landslide_event",
        "dist_to_source_area"
    ]
    houses_gpkg_tpl = str(houses_gpkg_tpl)
    houses_layer_tpl = str(houses_layer_tpl)

    for region in regions:
        houses_gpkg = Path(houses_gpkg_tpl.format(region=region))
        houses_layer = houses_layer_tpl.format(region=region)
        train_ds, val_ds, stats_path, h_train, h_val = make_region_dataset(
            region=region,
            houses_gpkg=houses_gpkg,
            houses_layer=houses_layer,
            tabular_cols=tab_cols,
            label_col="hazard_class_flom",
            patch_size_pixels=20,   # 200m @10m
            processed_raster_dir="../processed/rasters",
            stats_out_dir="../processed/channel_stats",
            stats_sample_n=1500,
            normalize=True,
            val_size=0.2,
            stratify=True,
        )

        train_loader = DataLoader(
            train_ds,
            batch_size=batch_size,
            shuffle=shuffle_train,
            num_workers=num_workers,
            pin_memory=pin_memory,
        )

        val_loader = DataLoader(
            val_ds,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=pin_memory,
        )

        results[region] = {
            "train_ds": train_ds,
            "val_ds": val_ds,
            "train_loader": train_loader,
            "val_loader": val_loader,
            "stats_path": stats_path,
            "houses_train": h_train,
            "houses_val": h_val,
        }

        print(
            f"[{region}] "
            f"train={len(train_ds)} val={len(val_ds)} "
            f"stats={Path(stats_path).name}"
        )

    return results


In [ ]:
bundle = build_all_regions_train(
    houses_gpkg_tpl="../raw/vector/houses/houses_{region}.gpkg",
    houses_layer_tpl="houses_{region}",
    batch_size=8,
)

In [ ]:
import pandas as pd
import torch

def concat_class_counts(concat_ds, label_col="hazard_class_flom"):
    """
    Build class counts from a ConcatDataset by reading each sub-dataset's df.
    Supports ds.houses_df or ds.df.
    """
    series_list = []
    for ds in concat_ds.datasets:
        if hasattr(ds, "houses"):
            series_list.append(ds.houses[label_col])
        elif hasattr(ds, "df"):
            series_list.append(ds.df[label_col])
        else:
            raise AttributeError("Sub-dataset missing houses_df/df for label counts.")

    labels = pd.concat(series_list, ignore_index=True)
    return labels.value_counts()

def make_class_weights_from_counts(class_counts, num_classes=3):
    """
    Simple inverse-frequency-ish weights.
    """
    weights = torch.ones(num_classes, dtype=torch.float32)
    total = class_counts.sum()

    for c in range(num_classes):
        if c in class_counts.index and class_counts[c] > 0:
            weights[c] = total / (num_classes * class_counts[c])

    weights = weights / weights.mean()
    return weights

def unpack_batch(batch, device):
    """
    Supports:
      - dict batch: {"patch": ..., "label": ...}
      - tuple batch: (x, y)
    """
    if isinstance(batch, dict):
        x = batch["patch"].to(device)
        y = batch["label"].to(device)
        return x, y

    if isinstance(batch, (list, tuple)) and len(batch) >= 2:
        x = batch[0].to(device)
        y = batch[1].to(device)
        return x, y

    raise ValueError("Unknown batch format. Expected dict or (x, y).")


In [ ]:
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from torch.optim import Adam
import pandas as pd


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

regions = ["sogn", "lillehammer", "oslo"]

all_train = ConcatDataset([bundle[r]["train_ds"] for r in regions])
all_val   = ConcatDataset([bundle[r]["val_ds"] for r in regions])

train_loader = DataLoader(all_train, batch_size=8, shuffle=True)
val_loader   = DataLoader(all_val, batch_size=8, shuffle=False)
model = SmallFloodCNN(in_channels=6, num_classes=3).to(device)

class_counts = concat_class_counts(all_train, label_col="hazard_class_flom")
weights = make_class_weights_from_counts(class_counts, num_classes=3).to(device)
criterion = nn.CrossEntropyLoss(weight=weights.to(device))
optimizer = Adam(model.parameters(), lr=1e-3)


In [ ]:
import torch
import torch.nn as nn

def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch in loader:
        x, y = unpack_batch(batch, device)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

    avg_loss = running_loss / max(1, len(loader))
    acc = correct / max(1, total)
    return avg_loss, acc


@torch.no_grad()
def evaluate(model, loader, criterion, device, return_preds=False):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    all_preds = []
    all_targets = []

    for batch in loader:
        x, y = unpack_batch(batch, device)

        logits = model(x)
        loss = criterion(logits, y)

        running_loss += loss.item()
        preds = logits.argmax(dim=1)

        correct += (preds == y).sum().item()
        total += y.size(0)

        if return_preds:
            all_preds.append(preds.detach().cpu())
            all_targets.append(y.detach().cpu())

    avg_loss = running_loss / max(1, len(loader))
    acc = correct / max(1, total)

    if return_preds:
        return avg_loss, acc, torch.cat(all_preds), torch.cat(all_targets)

    return avg_loss, acc


In [ ]:
for epoch in range(1, 21):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc, preds, targets = evaluate(
    model, val_loader, criterion, device, return_preds=True
    )

    # preds/targets are torch tensors on CPU
    torch.save(
        {
            "preds": preds,
            "targets": targets,
            "val_loss": val_loss,
            "val_acc": val_acc,
        },
        "val_predictions.pt"
    )
    print(
        f"Epoch {epoch:02d} | "
        f"train_loss={train_loss:.4f}, train_acc={train_acc:.3f} | "
        f"val_loss={val_loss:.4f}, val_acc={val_acc:.3f}"
    )